In [2]:
import pytorch_lightning as pl
import torch
import Operator as op
import utils
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


Setting up the Hamiltonian

In [3]:
lattice_sites = 3
hamiltonian = []
h = -1
'''
for l in range(lattice_sites):
  hamiltonian = Sx(l)* (h) + Sz(l) * Sz((l+1) % lattice_sites) + hamiltonian
print_op_list(hamiltonian)
'''
hamiltonian = op.Sx(2) + ([op.Sz(1)*op.Sy(2)] + hamiltonian)
op.print_op_list(hamiltonian)

adding to sequence
Hamiltonan = Sx_2 + Sz_1 * Sy_2


In [9]:
#setting up the datamodule
class spin_data(torch.utils.data.Dataset):
    def __init__(self, lattice_sites):
        self.spins = utils.get_all_spin_configs(lattice_sites)

    def __len__(self):
        return self.spins.shape[0]

    def __getitem__(self, index):
        return self.spins[index, :]

    def cuda(self):
        self.spins = self.spins.to(device)

data = spin_data(lattice_sites)
data.cuda()


In [4]:
import torch
from torch.nn import functional as F
import pytorch_lightning as pl
from torch import nn

class Model(pl.LightningModule):

  def __init__(self, lattice_sites):
    super().__init__()
    self.lattice_net = nn.Sequential(
      nn.Conv1d(1, 8, kernel_size=2, padding=1, padding_mode='circular'),
      nn.ReLU(),
      nn.Conv1d(8, 16, kernel_size=2, padding=1, padding_mode='circular'),
      nn.Flatten(start_dim=1, end_dim=-1)
    )
    
    self.tNN = nn.Sequential(
      nn.Linear(2, 16),
      nn.ReLU(),
      nn.Linear(16, 32),
      nn.ReLU(),
      nn.Linear(32, 64)
    )

    self.psi = nn.Sequential(
      nn.Linear( 64 + 16 * ( lattice_sites + 2 ), 256 ),
      nn.ReLU(),
      nn.Linear(256, 128),
      nn.ReLU(),
      nn.Linear(128,1)
    )

  def forward(self, spins, alpha):
    '''
    Forward function of the neural net to calculate psi. uses same spin config for all alpha values
    Parameters
    __________
    spins: tensor, dtype=int
      tensor of input spins to wave function 
      size = (num_spin_configs, lattice_sites)
    alpha: tensor, dtype=float
      other inputs to hamiltonian e.g. (time, ext_param) 
      size = (num_alpha_configs, num_inputs)

    Returns
    _______
    psi: tensor, dtype=complex
      wave function for a combination of (spins, alpha) 
      size = (num_spin_configs, num_alpha_configs)
    '''
    #circular padding needs tensor of dim 3
    spin_inp = torch.unsqueeze(spins, 1)
    lat_out = self.lattice_net(spin_inp)
    
    t_out = self.tNN(alpha)
    
    #now calculate psi for all combinations of t_out and lat_out
    #TODO make this less ugly!
    #getting all combinations of spin configs and alpha
    lat_out_batched = torch.flatten(torch.broadcast_to(lat_out, (t_out.shape[0], lat_out.shape[0], lat_out.shape[1])), end_dim=-2)
    t_out_batched = torch.flatten(torch.broadcast_to(t_out.unsqueeze(1), (t_out.shape[0], lat_out.shape[0], t_out.shape[1])), end_dim=-2)

    wave_fun = (self.psi(torch.cat((lat_out_batched, t_out_batched), dim=1))).reshape(t_out.shape[0], lat_out.shape[0], 1)
    return wave_fun
  
  def training_step(self, batch, batch_idx):
    #get psi(s, alpha)

    #get map and s' to the s from batch_idx

    #get psi(s', alpha)

    #calc dt_psi(s, alpha)

    #get mat_els for all alphas

    #get map, s' mat_els for O_loc_init

    #calc loss
    pass

    


    


model = Model(lattice_sites)

num_alpha_samples = 6
num_spin_samples = 5
spin_inp = torch.arange(1, lattice_sites+1, dtype=torch.float32).broadcast_to(num_spin_samples, lattice_sites)
alpha_arr = torch.ones(( num_alpha_samples, 2))
print(model(spin_inp, alpha_arr))

tensor([[[-0.0398],
         [-0.0398],
         [-0.0398],
         [-0.0398],
         [-0.0398]],

        [[-0.0398],
         [-0.0398],
         [-0.0398],
         [-0.0398],
         [-0.0398]],

        [[-0.0398],
         [-0.0398],
         [-0.0398],
         [-0.0398],
         [-0.0398]],

        [[-0.0398],
         [-0.0398],
         [-0.0398],
         [-0.0398],
         [-0.0398]],

        [[-0.0398],
         [-0.0398],
         [-0.0398],
         [-0.0398],
         [-0.0398]],

        [[-0.0398],
         [-0.0398],
         [-0.0398],
         [-0.0398],
         [-0.0398]]], grad_fn=<ViewBackward>)


In [5]:
from timeit import default_timer as timer

map = utils.get_map(hamiltonian, lattice_sites)
map = map.to(device)
print("map: ", map)

mat_els = utils.get_total_mat_els(hamiltonian, lattice_sites)
mat_els = mat_els.to(device)
print("mat els: ", mat_els)

#1.dim: Batch
#2.dim: lattice sites
#s_config = get_all_spin_configs(lattice_sites)
s_config = torch.tensor([1,1,1]).reshape(1,3)
s_config = (s_config.type(torch.float32)).to(device)
print("spin config: ", s_config)

start = timer()
s_p = utils.get_sp(s_config, map)
psi_s = model(s_config)
psi_sp = model(s_p.reshape(-1, lattice_sites)).reshape(s_p.shape[0], s_p.shape[1])
print(psi_sp.shape, psi_s.shape, s_config.shape)
O_loc = utils.calc_Oloc(psi_sp, mat_els, s_config)
end = timer()

print(f"time to calculate O_loc: {end - start:.2e}") 

map:  tensor([[[ 1,  1, -1],
         [ 1,  1, -1]]], device='cuda:0', dtype=torch.int8)
mat els:  tensor([[[[ 1.+0.j,  1.+0.j,  1.+0.j],
          [ 1.+0.j,  1.+0.j,  0.+1.j]],

         [[ 1.+0.j,  1.+0.j,  1.+0.j],
          [ 1.+0.j, -1.+0.j, -0.-1.j]]]], device='cuda:0')
spin config:  tensor([[1., 1., 1.]], device='cuda:0')


TypeError: forward() missing 1 required positional argument: 'alpha'

In [6]:
spins = torch.full((2,5), 0.5)
spins[1,:] /=4

alpha = torch.full((4,2), 1)
alpha[:, 1] = 2
alpha[:2, : ]*=4
spins_batched = torch.flatten(torch.broadcast_to(spins, (alpha.shape[0], spins.shape[0], spins.shape[1])), end_dim=-2)
alpha_batched = torch.flatten(torch.broadcast_to(alpha.unsqueeze(1), (alpha.shape[0], spins.shape[0], alpha.shape[1])), end_dim=-2)
print(alpha)
print(spins)

print(spins_batched)
print(alpha_batched)
#torch.cat((alpha, spins), dim=2)

tensor([[4, 8],
        [4, 8],
        [1, 2],
        [1, 2]])
tensor([[0.5000, 0.5000, 0.5000, 0.5000, 0.5000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])
tensor([[0.5000, 0.5000, 0.5000, 0.5000, 0.5000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250],
        [0.5000, 0.5000, 0.5000, 0.5000, 0.5000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250],
        [0.5000, 0.5000, 0.5000, 0.5000, 0.5000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250],
        [0.5000, 0.5000, 0.5000, 0.5000, 0.5000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])
tensor([[4, 8],
        [4, 8],
        [4, 8],
        [4, 8],
        [1, 2],
        [1, 2],
        [1, 2],
        [1, 2]])
